# PDil - Pipeline preprocessing

End-to-end example convertinng pt6305 raw data to nwb

In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from emu.pdil.raw import Electrophysiology,Participant,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from emu.pipeline.remote import RemoteCSV
from emu.pipeline.download import ExperimentManifest
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

from tqdm import tqdm_notebook as tqdm

In [2]:
all_files = ExperimentManifest(study='pdil').load()

In [3]:
seeg_root = os.path.expanduser('/home/elijahc/.emu/pdil/pt_6305/SEEG/raw')

nev_path = os.path.expanduser('/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.Events.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'PO_Day_02.*.ncs')))

# ncs = nlx.load_ncs()
print('nev path: ',nev_path)
print('ncs_paths:')
for p in ncs_paths[:10]:
    print(p)

nev path:  /home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.Events.nev
ncs_paths:
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC100_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC101_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC102_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC103_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC104_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC105_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC106_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC107_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC108_0007.ncs
/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC109_0007.ncs


In [4]:
p = Participant(patient_id=6305,raw_files=all_files,seeg_raw_path=seeg_root)

The [Participant](https://github.com/elijahc/emu/blob/3b240fbe8bfd4363ceadcf152dc6811c63493a3a/emu/pdil/raw.py#L221) class provides helper functions for managing all the raw data collected from a single patient

`Participant.cache_behavior()` returns a list of BehaviorRaw luigi tasks to fetch every raw behavior data.

- `BehaviorRaw.output().path` stores the path to where its file *should* exist locally.
- `BehaviorRaw.output().exists()` with retrn True if the file specified by `path` exists.

```python
def cache_behavior(self,verbose=False):
    for i,row in self.behavior_files.iterrows():
        t = BehaviorRaw(
            patient_id=row.patient_id,
            file_id=row.id,
            file_name=row.filename,
            save_to=self.behavior_raw_path,
        )
        yield t
```

- `Participant.load_game_data()` and `Participant.load_pdil_events()` will load mat files containing outcomes from the [pdil game](https://github.com/elijahc/emu/tree/master/PDil) implemented in psych toolbox as well as tic-toc timing of each screen and keypress which we'll sync to ephys data.

- `Participant.load_pdil_events()` and `Participant.load_game_data()` are generators that both use `cache_behavior()` to ensure all behavior files have been downloaded before trying to extract data from them.

```python
def load_pdil_events(self):
    tasks = list(self.cache_behavior())
    missing_tasks = [t for t in tasks if not t.output().exists()]
    print('{} missing tasks'.format(len(missing_tasks)))

    if len(missing_tasks) > 0:
        luigi.build(missing_tasks,local_scheduler=local_scheduler)

   ...
```
 
 - Both behavior load functions are python generators which will `yield` a pandas DataFrame for each block
 
 - If you wrap the function call in pd.concat(), to get a combined DataFrame across all blocks

In [5]:
p.behavior_files

,Unnamed: 0,filename,id,type,folder,patient_id
749,749,pt01_blockNum_1_computerTT_coop_taskoutput.mat,628721484958,Behavior,pt_01_postOpDay2_PDil,6305
750,750,pt01_blockNum_2_humanTT_coop_taskoutput.mat,628728186668,Behavior,pt_01_postOpDay2_PDil,6305
751,751,pt01_PRACTICE_blockNum_0 _taskoutput.mat,628698648907,Behavior,pt_01_postOpDay2_PDil,6305
754,754,pt01_blockNum_3_computerTT_defect_taskoutput.mat,629615908637,Behavior,pt_01_postOpDay4_PDil,6305
755,755,pt01_blockNum_4_humanTT_defect_taskoutput.mat,629620274728,Behavior,pt_01_postOpDay4_PDil,6305
756,756,pt01_blockNum_5_humanTT_coop_taskoutput.mat,629611713944,Behavior,pt_01_postOpDay4_PDil,6305
757,757,pt01_blockNum_6_computerTT_coop_taskoutput.mat,629611688205,Behavior,pt_01_postOpDay4_PDil,6305


In [6]:
pd.concat(p.load_pdil_events(local_scheduler=True)).head()

0 missing tasks


,event,event_delta,screen,trial,block,ttl_delta
0,trial_start,0.000000,NaN,0,1,0.000000
1,render_screen1,1.568142,1,1,1,1.568142
2,keypress1,1.951420,1,1,1,3.519562
3,render_screen2,1.564397,2,1,1,5.083959
4,keypress2,2.274455,2,1,1,7.358413


- `Participant.cache_nev()` and `Participant.cache_ncs()` are the sEEG analogues of `cache_behavior()` and basically do the same thing for the neuralynx channel files (.ncs) which store lfp traces and event files (.nev) which store timestamped ttls sent by the pdil task.

In [7]:
pd.concat(p.load_pdil_events(local_scheduler=True))

0 missing tasks


,event,event_delta,screen,trial,block,ttl_delta
0,trial_start,0.000000,NaN,0,1,0.000000
1,render_screen1,1.568142,1,1,1,1.568142
2,keypress1,1.951420,1,1,1,3.519562
3,render_screen2,1.564397,2,1,1,5.083959
4,keypress2,2.274455,2,1,1,7.358413
...,...,...,...,...,...,...
146,keypress3,1.901004,3,15,6,222.419018
147,render_screen4,1.548549,4,15,6,223.967568
148,keypress4,1.110800,4,15,6,225.078367
149,render_screen5,1.588783,5,15,6,226.667150


In [8]:
# Create a list of download tasks for the POD2 ncs files
# These files have a _0007 
d2_ncs_tasks = [t for t in p.cache_ncs() if 'PO_Day_02' in t.file_name and '0007' in t.file_name]
d2_ncs_paths = [t.output().path for t in d2_ncs_tasks]

print([t.file_name for t in d2_ncs_tasks[:10]],'\n')


# Create a list of download tasks for the POD4 nev files
d2_nev = [t for t in p.cache_nev() if 'PO_Day_02' in t.file_name and '0007' in t.file_name][0]
print(d2_nev)

['PO_Day_02.CSC100_0007.ncs', 'PO_Day_02.CSC101_0007.ncs', 'PO_Day_02.CSC102_0007.ncs', 'PO_Day_02.CSC103_0007.ncs', 'PO_Day_02.CSC104_0007.ncs', 'PO_Day_02.CSC105_0007.ncs', 'PO_Day_02.CSC106_0007.ncs', 'PO_Day_02.CSC107_0007.ncs', 'PO_Day_02.CSC108_0007.ncs', 'PO_Day_02.CSC109_0007.ncs'] 

NLXRaw(file=/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.Events_0007.nev)


In [9]:
luigi.build(d2_ncs_tasks+[d2_nev],workers=4)

True

In [10]:
[p for p in d2_ncs_paths if 'CSC105' in p]

['/home/elijahc/.emu/pdil/pt_6305/SEEG/raw/PO_Day_02.CSC105_0007.ncs']

In [11]:
p.electrode_locations

,chan_num,electrode,wire_num,anat_sh,anat_lg
0,1,1,1,LOF,L.Orbital.Frontal
1,2,2,1,LOF,L.Orbital.Frontal
2,3,3,1,LOF,L.Orbital.Frontal
3,4,4,1,LOF,L.Orbital.Frontal
4,5,5,1,LOF,L.Orbital.Frontal
...,...,...,...,...,...
173,174,4,15,RPT,R.Par.Temp
174,175,5,15,RPT,R.Par.Temp
175,176,6,15,RPT,R.Par.Temp
176,177,7,15,RPT,R.Par.Temp


In [12]:
from emu.nwb import ncs_to_nwb_raw

In [15]:
# nwb = p.create_nwb(d2_nev.output().path,d2_ncs_paths,blocks=[0,1,2],desc='Patient 1 | Post-op Day 2')
nwb = ncs_to_nwb_raw(d2_ncs_paths, nev_fp=d2_nev.output().path, 
                     electrode_locations=p.electrode_locations, 
                     desc='[RAW] Patient 1 | Post-op Day 2',
                     fs=4000,)

compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 15/15 [00:10<00:00,  1.43it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 10/10 [00:06<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 12/12 [00:07<00:00,  1.52it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 16/16 [00:10<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 10/10 [00:06<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 10/10 [00:06<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 8/8 [00:05<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 16/16 [00:10<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|██████████| 16/16 [00:10<00:00,  1.56it/s]
compressing channels to <class 'numpy.float16'> 4000Hz: 100%|█████

In [14]:
adf
nwb

NameError: name 'adf' is not defined

In [ ]:
nwb.trials.to_dataframe().head()

In [ ]:
nwb.processing.get('ecephys').data_interfaces.keys()

In [ ]:
for n in nwb.processing.get('ecephys').data_interfaces.keys():
    d = nwb.processing.get('ecephys').get(n).data
    if len(np.unique([len(dd) for dd in d])) < 2:
        print(np.stack(d).shape)

In [ ]:
with NWBHDF5IO(os.path.join('/home/elijahc/.emu/pdil/pt_6305/SEEG','PO_Day_02_raw.nwb'),'w') as io:
    io.write(nwb)

/home/elijahc/.pyenv/versions/emu_p38/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'ElectricalSeries/electrodes': Value with data type uint32 is being converted to data type int32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


In [ ]:
from emu.luigi.box import BoxClient

In [ ]:
def get_channel_id(fp):
    fn = fp.split('CSC')[1]
    if '_' in fn:
        return int(fn.split('_')[0])
    else:
        return int(fn.split('.ncs')[0])

In [ ]:
ephys = nwb.processing.get('ecephys')
ephys.get('LFP').electrical_series.get('wire_9_electrode_6').data

In [ ]:
import scipy.io as sio

In [ ]:
channels = [ch for ch in nwb.acquisition.keys() if ch.startswith('channel') or ch.startswith('wire')]

In [ ]:
[(nwb.acquisition[c].data.shape,c) for c in channels]

In [ ]:
def nwb_to_mat(out_mat,compress=True):
#     channels = [ch for ch in nwb.acquisition.keys() if ch.startswith('channel')]
    mod = nwb.processing.get('ecephys')
    
    mdict = {k:mod[k].data for k in mod.data_interfaces.keys()}
    sio.savemat(out_mat,mdict,do_compression=compress)


In [ ]:
md = nwb_to_mat('/home/elijahc/.emu/pdil/pt_6305/SEEG/processed/PO_Day_02.mat')


In [ ]:
from emu.luigi.box import BoxClient,file_id_to_path
box = BoxClient()

In [ ]:
file_id_to_path(633031167652)

In [ ]:
mat_path = '/home/elijahc/.emu/pdil/pt_01/SEEG/processed/PO_Day_02.mat'
nwb_path = '/home/elijahc/.emu/pdil/pt_01/SEEG/processed/PO_Day_02.nwb'
box.upload('/EMU/STUDY_PDil/PT_01/SEEG/processed',mat_path)

In [ ]:
from emu.pipeline.remote import RemoteCSV
RemoteCSV(file_path='/EMU/STUDY_PDil/PT_01/SEEG/electrode_locations.csv').load().head()